In [1]:
from __future__ import print_function

import requests
from requests_oauthlib import OAuth1

from urlparse import urlparse, urlunparse
from urllib import urlencode
from urllib import __version__ as urllib_version

import simplejson as json

import time
from datetime import datetime
import pytz

import sys

utc = pytz.utc

The following oauth keys have been disabled and are here as an example only.

In [2]:
credentials = {
'consumer_key': 'rAVG6KNhMYFmdlMKH9o',
'consumer_secret': '7uQA2s4YQrsrqPZp2xhujh54tWXBD4wRtyxEV6y7kHl',
'access_token_key': '144654323-rumbBx4JCgwQQUArpRZ6XJSyigjhjukkkYeERoh9qm9oN1',
'access_token_secret': 'Ihkn61Qfy0X6yaMbhbyzTEjNFfhdtWTKw70qyEOtTESQfdfdfN'}

### Recent 200 Followers of a Twitter ID

The following class uses the requests and requests_oauthlib packages to issue an html request to the Twitter REST API. It returns information about the 200 users to most recently follow a person/id on Twitter.

In [4]:
class RecentFollowers(object):
    """
    Class to query Twitter for 200 most recent followers of a user.
    """
    def __init__(self, user_id):
        """
        Initialize the class. Bring in and set config vars.
        """
        self._user_agent = 'Python-urllib/%s' % urllib_version
        self._url = 'https://api.twitter.com/1.1/followers/list.json'
        self._oauth = OAuth1(
            credentials['consumer_key'],
            credentials['consumer_secret'],
            credentials['access_token_key'],
            credentials['access_token_secret'])
        self._parameters = {
            'user_id': user_id,
            'count': int(200),
            'skip_status': False,
            'include_user_entities': True,
            'cursor': -1
        }

    def get(self):
        """
        Issue the get request to the Twitter API and parse the response.
        """
        # Parse the URL into components        
        (scheme, netloc, path, params, query, fragment) = urlparse(self._url)
        # Use parameters to encode request and add to URL
        encoded_params = urlencode(
            dict(
                [(k, str(v).encode('utf-8')) for k, v in list(self._parameters.items())]
            )
        )
        if query:
            query += '&' + encoded_params
        else:
            query = encoded_params
        # Form the URL
        url = urlunparse((scheme, netloc, path, params, query, fragment))
        # Issue the request
        resp = requests.get(url, auth=self._oauth)
        # Get the response code
        resp_code = resp.status_code
        # Get the data
        resp_data = resp.content.decode('utf-8')

        # Return a tuple of response code and data
        return (resp_code, resp_data)

### Execute

Notice that there are two components in the get method: the response code and the response data. If the code is 200, you have received a valid response. Otherwises, it's probably time to debug. The data is a single JSON blob.

In [7]:
# Hillary's user id is 1339835893
hillary_followers = RecentFollowers(user_id=1339835893)
(resp_code, resp_data) = hillary_followers.get()
if resp_code == 200:
    print("data loaded")
json_data = json.loads(resp_data)
print("data parsed")

data loaded
data parsed


In [11]:
len(json_data['users'])

200

### Streaming Filter API
The class below and example show how to use the streaming filter API. This is generally best run outside of Jupyter, as the stream process will block the kernel. I tend to collect data in a screened process to files on disk. You could use Spark Streaming to detect and load those files.

In [34]:
class Stream(object):
    """
    Class to query Twitter's streaming API.
    """
    def __init__(self, follow=None, track=None):
        """
        Initialize the class. Bring in and set config vars.
        """
        # Parse input options
        if follow is None and track is None:
            raise ValueError({'message': "follow or track filter must be specified."})
        # Get config
        self._user_agent = 'Python-urllib/%s' % urllib_version
        self._url = 'https://stream.twitter.com/1.1/statuses/filter.json'
        self._oauth = OAuth1(
            credentials['consumer_key'],
            credentials['consumer_secret'],
            credentials['access_token_key'],
            credentials['access_token_secret'])
        self._parameters = {
            'stall_warnings': str(True)
        }
        if follow:
            self._parameters['follow'] = ','.join(follow)
        if track:
            self._parameters['track'] = ','.join(track)
    
    def get(self):
        try:
            return requests.post(self._url, data=self._parameters, stream=True, auth=self._oauth, timeout=60)
        except Exception as e:
            print("exception: %s" % str(e))

In [35]:
stream_data = Stream(track=['data','spark'])

#### Streamed Data
Each line is a response, but you need to use iter_lines on the returned data to form the responses into complete lines. Otherwise, entries may be split between lines. Also, there are no commas between responses, just line breaks.

In [39]:
for line in stream_data.get().iter_lines():
    if line:
        print('\nstart --%s-- end\n' % line)


start --{"created_at":"Tue May 17 02:42:29 +0000 2016","id":732400871733088256,"id_str":"732400871733088256","text":"#GARMIN Power\/data Cable, MFG#010-10083-00, Used With GPS-126\/128\/152 and GPSMAP-162\/182\/182C. \/ GA-0101008300 https:\/\/t.co\/aaoWPiIUHr","source":"\u003ca href=\"http:\/\/socialsharinglink.com\" rel=\"nofollow\"\u003eCreate Deal Alerts for PC Access\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":4798505853,"id_str":"4798505853","name":"PC Accessory Deals","screen_name":"pcaccessoryusa","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":144,"friends_count":0,"listed_count":160,"favourites_count":0,"statuses_count":101023,"created_at":"Wed Jan 13 23:11:06 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_tra

KeyboardInterrupt: 